In [1]:
#### IMPORTING DEPENDENCIES ####

import numpy as np
import pandas as pd
from lightfm import LightFM


/anaconda/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
#### IMPORTING DATASETS ####

training_set = pd.read_csv('jester_train.csv')
test_set = pd.read_csv('jester_test.csv')

In [3]:
#### SANITY CHECK ####

training_set.head()

,user_id,joke_id,rating
0,7302,29,7.156
1,61815,46,6.375
2,31128,96,2.281
3,36125,147,-1.781
4,18007,60,2.188


In [4]:
#### CONVERTING THE DATA INTO 2D MATRIX ####

data, row, col = [], [], []
jokes, users = {}, {}

In [5]:
for n in range(0, 100000):
    user = int(training_set.user_id[n])
    joke = int(training_set.joke_id[n])
    rating = float(training_set.rating[n])
    
    if user not in users:
        users[user] = len(users)

    if joke not in jokes:
        jokes[joke] = len(jokes)
        
    data.append(rating)
    row.append(users[user])
    col.append(jokes[joke])            

In [6]:
#### SPARSING THE MATRIX ####

from scipy.sparse import coo_matrix

coo = coo_matrix((data,(row,col))).tocsr()

In [7]:
#### CLEANING TEST DATA (OPTIONAL) ####

test_user, test_joke = [], []

testusers, testjokes = {}, {}
for n in range(len(test_set.user_id)):
    user = int(test_set.user_id[n])
    joke = int(test_set.joke_id[n])
    
    if user not in testusers:
        testusers[user] = len(testusers)

    if joke not in testjokes:
        testjokes[joke] = len(testjokes)
        
    test_user.append(testusers[user])
    test_joke.append(testjokes[joke])      

In [8]:
#### THE MODEL ####

model = LightFM(loss='warp')
model.fit(coo, epochs=30, num_threads=2)

In [9]:
n_items = coo.shape[1]

In [12]:
#### PREDICTING ####

def get_recommendations(model, coo_mtrx, users_ids):
    
    for user in users_ids:

        # TODO create known positives
        
        # Artists the model predicts they will like
        scores = model.predict(user, np.arange(n_items))
        top_scores = np.argsort(-scores)[:3]

        print('Recomendations for user %s:' % user)

        for x in top_scores[:3]:
            print("Joke Number: %s" % x)
           
            
            

In [13]:
#### RESULT ####

get_recommendations(model, coo, [4, 50,30])

Recomendations for user 4:
Joke Number: 83
Joke Number: 6
Joke Number: 90
Recomendations for user 50:
Joke Number: 49
Joke Number: 14
Joke Number: 74
Recomendations for user 30:
Joke Number: 86
Joke Number: 37
Joke Number: 99
